# Organ microbiomes of two *Biomphalaria* snail species.


## Aim

We previously characterized the hemolymph (blood) microbiome of several *Biomphalaria* snail species ([Chevalier *et al.*, 2020](https://doi.org/10.1111/1462-2920.15303)) and showed that these microbiomes were diverse. Since hemolymph bathes snail organs, our current study aims to investigate the composition of the microbiome of these organs to determine if they exist and, if so, whether they are similar to the hemolymph or specific to the organs. We dissected ten snails for each of two different species (*B. alexandrina* and *B. glabrata*) and collected their hemolymph and organs (ovotestis, hepatopancreas (i.e., liver), gut, and stomach). We also ground four whole snails of each species in liquid nitrogen. Additionally, we sampled the water in which the snails were living (environmental controls). We sequenced the 16S V4 rDNA of each sample. In this notebook, we detail the analysis done to characterize the different sample microbiomes and compare them. The results are presented in the manuscript entitled "How should we sample snail microbiomes?".

Several folders are are already available or will be created during the analysis. Here is the list of the folders and their contents:
* **data**: Sequencing files, qPCR result file, database for taxonomy assignment, sample metadata, and list of contaminants. These files will be either present or downloaded.
* **.env**: Files needed to create appropriate environment.
* **graphs**: Graphical representation of the data. If not existing, this will be created during the analysis.
* **results**: Files generated through data processing. If not existing, this will be created during the analysis.
* **scripts**: Scripts used for the analysis.



## Environment and data

### Creating environment

Creating a conda environment improves reproducibility by installing specific versions of the programs used.

In [ ]:
# Check if conda available
[[ ! $(which conda 2> /dev/null) ]] && echo "conda not available in \$PATH. Please interrupt the kernel and fix the situation." && sleep inf

# Creating conda environment
conda env create -f .env/env.yml

This cell must be run each time a new session of Jupyter is run.

In [ ]:
# Activate the environment
source $(sed "s,/bin/conda,," <<<$CONDA_EXE)/etc/profile.d/conda.sh
conda activate ubiome_organs

# Remove potential variable interferences
export PERL5LIB=""
export PYTHONNOUSERSITE=1

In [ ]:
# Installing needed R packages
Rscript ".env/R package dependencies.R"

# Because of issues with devtools, this script will download sources and install packages from outside R
bash ".env/R package dependencies.sh"

### Downloading sequencing data

This step downloads the fastq files of the different samples.

In [ ]:
# Data directory
ldir="data/libraries"
[[ ! -d "$ldir" ]] && mkdir -p "$ldir"

# Bioproject
bioproject=PRJNA1113672

# Download related information to data project
runinfo=$(esearch -db sra -query ${bioproject} | efetch -format runinfo | sed "/^$/d")

# Field of interest (library name and weblink)
fdn=$(head -n 1 <<< "$runinfo" | tr "," "\n" | grep -w -n "LibraryName" | cut -d ":" -f 1)
fdr=$(head -n 1 <<< "$runinfo" | tr "," "\n" | grep -w -n "Run" | cut -d ":" -f 1)

# Download fastq files
while read line
do
    # Filename and download link
    fln=$(cut -d "," -f $fdn <<<$line)
    run=$(cut -d "," -f $fdr <<<$line)
    
    # Download
    echo "$fln"
    #wget -P "$ldir" -O "$fln" "http://trace.ncbi.nlm.nih.gov/Traces/sra/sra.cgi?cmd=dload&run_list=${run}&format=fastq"
    fastq-dump -O "$ldir" --split-files "$run"
    
    mv "$ldir/${run}_1.fastq" "$ldir/${fln}_R1.fastq"
    mv "$ldir/${run}_2.fastq" "$ldir/${fln}_R2.fastq"
        
done < <(tail -n +2 <<< "$runinfo")

# Compress files
pigz "$ldir/"*

### Downloading database

The Silva database is used to assign taxonomy to the ASVs generated from the sequencing data.

In [ ]:
# Database directory
dbdir="data/Silva db"
[[ ! -d "$dbdir" ]] && mkdir -p "$dbdir"


# Download and extract the relevant Silva file
wget -P "$dbdir" 'https://www.arb-silva.de/fileadmin/silva_databases/qiime/Silva_132_release.zip'
unzip "$dbdir/Silva_132_release.zip" -d "$dbdir" && rm "$dbdir/Silva_132_release.zip"

# Import the sequence database in Qiime format
qiime tools import \
    --input-path "$dbdir/SILVA_132_QIIME_release/rep_set/rep_set_16S_only/99/silva_132_99_16S.fna" \
    --output-path "$dbdir/silva_132_99_16S.qza" \
    --type 'FeatureData[Sequence]'

# Import the taxonomy database in Qiime format
qiime tools import \
    --input-path "$dbdir/SILVA_132_QIIME_release/taxonomy/16S_only/99/taxonomy_all_levels.txt" \
    --output-path "$dbdir/silva_132_99_16S_taxa.qza" \
    --type 'FeatureData[Taxonomy]' \
    --input-format HeaderlessTSVTaxonomyFormat

## Qiime pipeline

This section process the data to generate ASVs and assign taxonomy.

In [ ]:
# Qiime output directory
qdir="results/1-qiime"
[[ ! -d "$qdir" ]] && mkdir -p "$qdir"

# Metadata file
metadata="data/sample-metadata.tsv"

**Note**: C.4.BgBS90.O sample is missing because sample amplification failed. So not library was made from this sample.

In [ ]:
# Check for sequencing data
[[ ! $(find "$ldir" -type f -name *fastq.gz) ]] && echo  "No sequencing data. Please interrupt the kernel and fix the situation." && sleep inf

# Create the manifest for importing data in artefact
## source: https://docs.qiime2.org/2019.4/tutorials/importing/#fastq-manifest-formats
for i in $(ls "$ldir"/* | sed "s,_R[12].fastq.*,,g" | uniq)
do
    nm=$(sed "s,$ldir/,," <<<$i)
    fl=$(ls -1 "$PWD/$i"* | tr "\n" "\t")

    echo -e "$nm\t$fl"
done > "$qdir/manifest"

# Add header
sed -i "1s/^/sample-id\tforward-absolute-filepath\treverse-absolute-filepath\n/" "$qdir/manifest"

# Import data
## source: https://docs.qiime2.org/2019.4/tutorials/importing/
qiime tools import \
    --type 'SampleData[PairedEndSequencesWithQuality]' \
    --input-path "$qdir/manifest" \
    --input-format PairedEndFastqManifestPhred33V2 \
    --output-path "$qdir/demux-paired-end.qza"

**Note**:
* No need to remove adapters and barcodes. This has been done during `bcl2fastq`. This can be checked using `grep`.
* Importing with `--input-format PairedEndFastqManifestPhred33V2` instead of `--input-format CasavaOneEightSingleLanePerSampleDirFmt` for [custom sample names](https://docs.qiime2.org/2019.4/tutorials/importing/#fastq-manifest-formats).

### Data quality

To assess data quality, we need to generate a visualization to check data quality. The visualization can be view on [Qiime2 website](https://view.qiime2.org/)

In [ ]:
# Make a summary to check read quality
qiime demux summarize \
    --i-data "$qdir/demux-paired-end.qza" \
    --o-visualization "$qdir/demux-paired-end.qzv"

### Sequence clustering and denoising

This steps generates ASVs from the sequencing data. This step is perform by the `dada2` module.

In [ ]:
qiime dada2 denoise-paired \
    --i-demultiplexed-seqs "$qdir/demux-paired-end.qza" \
    --p-trunc-len-f 177 \
    --p-trunc-len-r 202 \
    --p-trim-left-f 0 \
    --p-trim-left-r 13 \
    --p-max-ee-f 5 \
    --p-max-ee-r 10 \
    --p-n-threads 0 \
    --o-table "$qdir/table.qza" \
    --o-representative-sequences "$qdir/rep-seqs.qza" \
    --o-denoising-stats "$qdir/denoising-stats.qza"

In [ ]:
# Add metadata information to the denoising stats
qiime metadata tabulate \
    --m-input-file "$metadata" \
    --m-input-file "$qdir/denoising-stats.qza" \
    --o-visualization "$qdir/denoising-stats.qzv"

### Taxonomy identification

This step assigns taxonomy to the ASVs generated. This is done using the Silva database and the `classify-consensus-vsearch` method from the `feature-classifier` plugin with an identity threshold of 97%.

In [ ]:
# Database directory
dbdir="data/Silva db"
[[ ! -d "$dbdir" ]] && mkdir -p "$dbdir"

qiime feature-classifier classify-consensus-vsearch \
    --i-query "$qdir/rep-seqs.qza" \
    --i-reference-reads "$dbdir/silva_132_99_16S.qza" \
    --i-reference-taxonomy "$dbdir/silva_132_99_16S_taxa.qza" \
    --p-perc-identity 0.97 \
    --p-threads $(nproc) \
    --o-classification "$qdir/rep-seqs_taxa.qza"

### Phylogeny

This step generates a phylogeny from the ASVs ([source](https://chmi-sops.github.io/mydoc_qiime2.html)). ASVs are aligned, masked for highly variable regions. A tree is created from the alignment and rooted in its center.

In [ ]:
# Multiple seqeunce alignment using Mafft
qiime alignment mafft \
    --i-sequences "$qdir/rep-seqs.qza" \
    --o-alignment "$qdir/aligned-rep-seqs.qza"

# Masking (or filtering) the alignment to remove positions that are highly variable. These positions are generally considered to add noise to a resulting phylogenetic tree.
qiime alignment mask \
    --i-alignment "$qdir/aligned-rep-seqs.qza" \
    --o-masked-alignment "$qdir/masked-aligned-rep-seqs.qza"

# Creating tree using the Fasttree program
qiime phylogeny fasttree \
    --i-alignment "$qdir/masked-aligned-rep-seqs.qza" \
    --o-tree "$qdir/unrooted-tree.qza"

# Root the tree using the longest root
qiime phylogeny midpoint-root \
    --i-tree "$qdir/unrooted-tree.qza" \
    --o-rooted-tree "$qdir/rooted-tree.qza"

## Library analysis

To better understand the impact of the different steps on the number of reads filtered out, we analyze the Qiime2 visualization generated after the denoising step. We generate a table summarizing number of reads retained (mean, standard deviation and range) at each step, which includes the number of initial, filtered, denoised, merged and non chimeric reads, for each population and replicate.

In [ ]:
Rscript scripts/library_stats.R

## Identification of ASVs with unassigned taxonomy

ASVs with unassigned taxonomy could correspond to eukaryotic contaminants because of the 16S primers amplifying on the 5S or 18S regions. To exclude such contaminants, we perform a megablast search againt the NCBI nt database to find the best sequence similarity to a given unassigned ASV. The results will be then used to exclude non 16S sequences for the subsequent analysis.

Because of the time this analysis can take (up to 7h), the list of contaminants is already available in the data folder.

### Export of unassigned ASV

We use Qiime to filter and export ASV without taxonomy assignments.

In [ ]:
# Filtering assigned ASV out 
qiime taxa filter-seqs \
    --i-sequences "$qdir/rep-seqs.qza" \
    --i-taxonomy "$qdir/rep-seqs_taxa.qza" \
    --p-include "Unassigned" \
    --o-filtered-sequences "$qdir/rep-seqs_unassigned.qza"

# Exporting sequences in fasta format
qiime tools export \
    --input-path "$qdir/rep-seqs_unassigned.qza" \
    --output-path "$qdir/rep-seqs_unassigned"

### Blast and annotation of unassigned ASVs

To identify the most similar sequences to the unassigned ASVs, we perform a megablast. This is done using a relatively lenient e-value parameter (1e-2) to increase power of detection. This step is relatively long (estimated running time: 5 h - 8 h) even when using a maxiumum target of 1 with a maximum alignment (HSPs) of 1. The resulting table is then updated with the title and the phyla of the blast match (estimated running time: 30 min - 1 h). Finally we correct the annotations because some ASVs match 16S mitochondrial DNA of eukayote organisms and are wrongly classified as eukaryotes while they likely represent bacteria.

In [ ]:
# Blast the unassigned against the nt database
blastn -task megablast -db nt -remote \
    -query "$qdir/rep-seqs_unassigned/dna-sequences.fasta" \
    -max_target_seqs 1 \
    -max_hsps 1 \
    -evalue 1e-2 \
    -outfmt 6 > "$qdir/rep-seqs_unassigned/unassigned.blastn.tsv"

# Complete the table to identify what kind of organism the match belongs to
for ((i=1; i <= $(wc -l < "$qdir/rep-seqs_unassigned/unassigned.blastn.tsv"); i++))
do
    # Get GI from the blast result
    gi=$(sed -n "${i}p" "$qdir/rep-seqs_unassigned/unassigned.blastn.tsv" | cut -f 2)
    
    # Download entry and get title and phylym info
    entry=$(wget -q -O - "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nuccore&id=${gi}&rettype=gb")
    title=$(echo "$entry" | grep "^DEFINITION" | cut -d " " -f 3-)
    phylum=$(echo "$entry" | grep -A 1 "^ *ORGANISM" | sed -n "2p" | cut -d ";" -f 1 | sed "s/ *//g")
    
    # Update table line
    sed -i "${i}s/$/\t$title\t$phylum/" "$qdir/rep-seqs_unassigned/unassigned.blastn.tsv"
    
    # Sleep a little to avoid server closing connection on next request
    sleep 0.25s
done

# Correct annotation
for i in $(egrep -n "mitochondri.*Eukaryota$" "$qdir/rep-seqs_unassigned/unassigned.blastn.tsv" | cut -d ":" -f 1)
do
    sed -i "${i}s/Eukaryota/Bacteria/" "$qdir/rep-seqs_unassigned/unassigned.blastn.tsv"
done

## Microbiome diversity

We measure several parameters to characterize the different microbiomes:
* **Rarefaction curves**: we generate them for each library to check whether sequencing effort was enough.
* **$\alpha$-diversity**: we measured observed richness, Faith's phylogenetic diversity and evenness and tested which factor(s) (species, sample type, etc.) explained the variation observed using linear mix-effect models.
* **$\beta$-diversity**: we measured unweighted and weighted UniFrac to understand how microbial communities were related.
* **Taxonomic diversity**: we investigated which taxa were found in sample types.
* **Shared taxa**: we quantified the proportion of shared taxa between organs, hemolymph and water.

Details about the methods used are available in the R script. Analysis of the results are detailed in the manuscript.

In [ ]:
Rscript scripts/microbiome_diversity.R

To generate the final version of Figure 5 and supp. figure 3, wee need to merge the figures (panels) into one. This can be done using `pdfjam` from TexLive package (not installed). The command line is as follows:

## Microbiome density

We investigate the bacterial density of each sample type, comparing either organs or hemolymph/water. Bacterial density was estimated using 16S qPCR. We compare quantities within each snail species and perform correlation to test if bacterial densities in different sample types show similar trends.

In [ ]:
# qPCR output directory
ddir="data/qPCR"
[[ ! -d "$ddir" ]] && echo "$ddir and qPCR data are missing. Please interrupt the kernel and fix the situation." && sleep inf

# Analyze data
Rscript scripts/microbiome_density.R